In [1]:
import pandas as pd
import geopandas as gpd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.colors as colors
import contextily as ctx
import json
import base64
from io import BytesIO
from pyproj import Transformer
from shapely.geometry import LineString, box

In [2]:
# Número de filas a leer del archivo CSV
nrows = 100000
# Ruta del archivo CSV
filename = "../train_data/taxis_trajectory/train.csv"

In [3]:
def load_and_simplify_data(filename, rows, tolerance=0.001, umbral_distancia=0.01):
    # Cargar datos
    df = pd.read_csv(filename, nrows=rows, sep=",", low_memory=False)
    
    # Filtrar y crear LineString para cada polilínea
    def create_line(x):
        points = json.loads(x)
        if len(points) > 1:
            return LineString(points)
        return None
    
    df['geometry'] = df['POLYLINE'].apply(create_line)
    
    # Eliminar filas con geometrías nulas
    df = df[df['geometry'].notnull()]
    
    # Convertir a Geopandas DataFrame
    gdf = gpd.GeoDataFrame(df, geometry='geometry')
    
    # Simplificar las geometrías
    gdf['geometry'] = gdf['geometry'].simplify(tolerance)
    
    return gdf

In [4]:
def filter_data_in_area(gdf, minx, miny, maxx, maxy):
    # Crear un polígono de área de interés
    area_of_interest = box(minx, miny, maxx, maxy)
    
    # Filtrar los datos para incluir solo aquellos completamente dentro del área de interés
    gdf_filtered = gdf[gdf.geometry.within(area_of_interest)]
    
    return gdf_filtered

In [5]:
def lisr_coordinates(gdf):   
    # Crear listas vacías para las coordenadas x e y
    x_coords_flat = []
    y_coords_flat = []

    # Iterar sobre cada geometría en el GeoDataFrame
    for geom in gdf['geometry']:
        # Verificar que la geometría sea una LineString
        if isinstance(geom, LineString):
            # Iterar sobre cada punto en la LineString
            for point in geom.coords:
                x_coords_flat.append(point[0])  # Añadir la coordenada x a la lista
                y_coords_flat.append(point[1])  # Añadir la coordenada y a la lista
    
    return x_coords_flat, y_coords_flat

In [6]:
def map_ilustration(gdf, minx, miny, maxx, maxy):
    gdf = gdf.set_crs("EPSG:4326")

    # Luego, usar estas coordenadas en la función de filtrado
    gdf = filter_data_in_area(gdf, minx, miny, maxx, maxy)

    # Transforcion en EPSG:3857 para alinear con el mapa base de Contextily
    gdf = gdf.to_crs(epsg=3857)
    
    # Crear una figura con Matplotlib
    fig, ax = plt.subplots(figsize=(10, 10), dpi=300)
    gdf.plot(ax=ax, linewidth=0.5, color='blue')

    # Añadir un mapa base con Contextily
    ctx.add_basemap(ax, source=ctx.providers.CartoDB.Positron)

    # Añadir título y etiquetas
    plt.title('Mapa de Trayectorias de Taxis con Mapa de Fondo')
    plt.xlabel('Longitud')
    plt.ylabel('Latitud')

    # Crear un objeto BytesIO para guardar la imagen
    img_data = BytesIO()
    plt.savefig(img_data, format='png')
    img_data.seek(0)  # Mover el 'cursor' al principio del archivo en memoria
    
    # Es importante cerrar la figura para liberar memoria
    plt.close(fig)

    # Codificar la imagen generada en base64
    encoded_string = base64.b64encode(img_data.read()).decode('utf-8')

    return encoded_string

In [7]:
def map_heat(gdf, minx, miny, maxx, maxy, bin_count, posicion_x, posicion_y, zoom):  
    # Obtener las coordenadas x e y de las geometrías 
    x_coords_flat, y_coords_flat = lisr_coordinates(gdf)

    # Calcular el histograma bidimensional de las coordenadas x e y
    heatmap, _, _ = np.histogram2d(x_coords_flat, y_coords_flat, bins=bin_count, density=True, range=[[minx, maxx], [miny, maxy]])

    # Inicializar el transformador de coordenadas
    transformer = Transformer.from_crs("epsg:4326", "epsg:3857", always_xy=True)

    # Transformar las coordenadas
    xmin, ymin = transformer.transform(minx, miny)
    xmax, ymax = transformer.transform(maxx, maxy)
    
    """ # Calcular el centro y el rango de los ejes x e y
    x_center, y_center = ((xmin + xmax) / 2) + posicion_x, ((ymin + ymax) / 2) + posicion_y
    x_range, y_range = (xmax - xmin) / zoom, (ymax - ymin) / zoom """

    # Crear la figura y los ejes para matplotlib
    fig, ax = plt.subplots(figsize=(10, 10), dpi=300)

    # Crear una normalización logarítmica
    norm = colors.LogNorm(vmin=heatmap.min()+1, vmax=heatmap.max())

    # Mostrar el mapa de calor y capturar el objeto mappable retornado por imshow
    mappable = ax.imshow(heatmap.T, origin='lower', norm=norm ,extent=[xmin, xmax, ymin, ymax], aspect='auto', alpha=0.7, zorder=2)

    # Añadir el mapa base
    ctx.add_basemap(ax, source=ctx.providers.CartoDB.Positron, zoom='auto')

    """ # Ajustar los límites de los ejes para coincidir con los bordes del histograma
    ax.set_xlim(x_center - x_range, x_center + x_range)
    ax.set_ylim(y_center - y_range, y_center + y_range) """

    # Añadir barra de color, títulos y etiquetas usando el objeto mappable
    plt.colorbar(mappable, label='Densidad')
    plt.title('Mapa de Calor de Trayectorias de Taxis con Mapa de Fondo')
    plt.xlabel('Longitud')
    plt.ylabel('Latitud')

    # Crear un objeto BytesIO para guardar la imagen
    img_data = BytesIO()
    plt.savefig(img_data, format='png')
    img_data.seek(0)  # Mover el 'cursor' al principio del archivo en memoria
    
    # Es importante cerrar la figura para liberar memoria
    plt.close(fig)

    # Codificar la imagen generada en base64
    encoded_string = base64.b64encode(img_data.read()).decode('utf-8')

    return encoded_string

In [8]:
def solicitar_coordenadas(gdf):
    """ print("Por favor, introduce las coordenadas para el área de interés.")
    minx = float(input("Introduce la longitud mínima (minx): "))
    miny = float(input("Introduce la latitud mínima (miny): "))
    maxx = float(input("Introduce la longitud máxima (maxx): "))
    maxy = float(input("Introduce la latitud máxima (maxy): ")) """
    
    """ minx=-8.689
    miny=41.107
    maxx=-8.560
    maxy=41.185 """

    x_coords_flat, y_coords_flat = lisr_coordinates(gdf)

    maxx, maxy, minx, miny = max(x_coords_flat), max(y_coords_flat), min(x_coords_flat), min(y_coords_flat)

    return minx, miny, maxx, maxy 

In [9]:
# Esto aun no se como pedirlo o si hacerlo
def solicitar_map_position():
    bin_count = 300 # Cantidad de bins para el histograma 2D
    posicion_x = 0 # Coordenada x del centro del mapa, para despazarlo por el eje y, poner valore arededro de 1000
    posicion_y = 0 # Coordenada y del centro del mapa
    zoom = 2 # Nivel de zoom del mapa, por cuanto se divide las cordenadas

    return bin_count, posicion_x, posicion_y, zoom

In [10]:
# Cargar y simplificar datos
gdf = load_and_simplify_data(filename, nrows)

In [11]:
# solicitar coordenadas
minx, miny, maxx, maxy = solicitar_coordenadas(gdf)

# solicitar datos del mapa
bin_count, posicion_x, posicion_y, zoom = solicitar_map_position()

In [12]:
# Crear un mapa ilustrativo
html_map = map_ilustration(gdf, minx, miny, maxx, maxy)

# Crear un mapa de calor
html_heatmap = map_heat(gdf, minx, miny, maxx, maxy, bin_count, posicion_x, posicion_y, zoom)

# A partir de aqui empieza la plagina web, todo lo anterio estara en un archivo diferente en la implementacion final

In [13]:
import dash
import dash_bootstrap_components as dbc
from dash import Input, Output, html, dcc, State

In [349]:
# Selecciona un tema de Bootstrap
app = dash.Dash(__name__, external_stylesheets=[dbc.themes.LUMEN])

# LUMEN, MORPH

In [350]:
def get_map_image_as_html(html_map, html_heatmap):
    return html.Div([
            html.Div([
                html.Img(
                    src=f"data:image/png;base64,{html_map}",
                    style={
                        'height': '600px',
                        'width': '600px',
                        'display': 'block',
                        'margin-left': 'auto',
                        'margin-right': 'auto'
                    }
                ),
            ], style={'display': 'flex', 'justifyContent': 'center', 'alignItems': 'center', 'paddingRight': '5px'}),
            html.Div([
                html.Img(
                    src=f"data:image/png;base64,{html_heatmap}",
                    style={
                        'height': '600px',
                        'width': '600px',
                        'display': 'block',
                        'margin-left': 'auto',
                        'margin-right': 'auto'
                    }
                ),
            ], style={'display': 'flex', 'justifyContent': 'center', 'alignItems': 'center', 'paddingLeft': '5px'})
        ], style={'display': 'flex', 'justifyContent': 'center'})

In [351]:
def get_home_page():
    return html.Div([
        dbc.NavbarSimple(
            html.Div([
            html.Div([
                dbc.Label("Choose latitud and longitud"),
                dbc.RadioItems(
                    options=[
                        {"label": "Option 1", "value": 1},
                        {"label": "Option 2", "value": 2},
                        {"label": "Option 3", "value": 3},
                    ],
                    value=1,
                    id="lat-long-radio-items",
                ), 
            ], style={'padding': '40px'}),
            html.Div([
                dbc.Label("Choose zoom and position of the map"),
                dbc.RadioItems(
                    options=[
                        {"label": "Option 1", "value": 1},
                        {"label": "Option 2", "value": 2},
                        {"label": "Option 3", "value": 3},
                    ],
                    value=1,
                    id="zoom-position-radio-items",
                ),
            ], style={'padding': '40px'}),
        ]),
        # brand_href="#",
        # color="pink",
        # dark=False
        ), 
        dbc.Col(width=9, children=[
            dcc.Loading(
                id="loading-map-1",
                type="default",
                children=html.Div(id="map-container")
            ),
        ]),
    ], style={'display': 'flex', 'justifyContent': 'space-around'})

In [352]:
def get_clusters_map():
    return html.Div([
        dbc.Carousel(
            items=[
                {"key": "1", "src": f"data:image/png;base64,{html_heatmap}"},
                {"key": "2", "src": f"data:image/png;base64,{html_map}"}
            ],
            controls=True,
            indicators=True,
            variant="dark"
        )
    ], style={'height': '600px', 'width': '600px', 'display': 'block', 'margin-left': 'auto', 'margin-right': 'auto'}) 

In [353]:
def get_comparation_page():
    items1 = [
        dbc.DropdownMenuItem("Item 1", id="item-1-1"), 
        dbc.DropdownMenuItem("Item 2", id="item-1-2"),
        dbc.DropdownMenuItem("Item 3", id="item-1-3")
    ]

    items2 = [
        dbc.DropdownMenuItem("Item 1", id="item-2-1"), 
        dbc.DropdownMenuItem("Item 2", id="item-2-2"),
        dbc.DropdownMenuItem("Item 3", id="item-2-3")
    ]

    return html.Div([
        html.Div([
            dbc.DropdownMenu(
                items1, label="Primary", color="primary", className="m-1"
            ),
            dbc.DropdownMenu(
                items2, label="Secondary", color="secondary", className="m-1"
            ),
        ], style={'display': 'flex', 'justifyContent': 'center'}),
        html.Div([
            dbc.Col(width=9, children=[
                dcc.Loading(
                    id="loading-cluster-1",
                    type="default",
                    children=html.Div(id="map-clusters-1")
                ),
            ]),
        ], style={'display': 'flex', 'justifyContent': 'center', 'alignItems': 'center', 'paddingRight': '5px'}),
        html.Div([
            dbc.Col(width=9, children=[
                dcc.Loading(
                    id="loading-cluster-2",
                    type="default",
                    children=html.Div(id="map-clusters-2")
                ),
            ]),
        ], style={'display': 'flex', 'justifyContent': 'center', 'alignItems': 'center', 'paddingRight': '5px'}),
    ], style={'display': 'flex', 'justifyContent': 'center'})

In [354]:
def get_estadistic_page():
    return html.Div([
        html.H1('Estadísticas'),
        html.P('Página de contacto.'),
    ])

In [355]:
app.layout = html.Div([
    dcc.Location(id='url', refresh=False),
    dbc.Navbar(
        dbc.Container(children=[
            html.Div([
                dbc.NavbarBrand("TRACLUS", style={'color': 'white', 'fontSize': '50px'}),
            ], style={'width': '100%', 'display': 'flex', 'justify-content': 'center'}),
            html.Div([
                dbc.DropdownMenu(
                    label="Menú",
                    children=[
                        dbc.DropdownMenuItem("Inicio", href="/"),
                        dbc.DropdownMenuItem("Comparación", href="/comparacion"),
                        dbc.DropdownMenuItem("Estadísticas", href="/estadisticas"),
                    ]
                ),
            ]),   
        ]),
        color="primary"
    ),
    html.Div(id='page-content')
])

In [356]:
@app.callback(
    Output('page-content', 'children'), 
    [Input('url', 'pathname'),]
)

def display_page(pathname):
    if pathname == '/comparacion':
        return  get_comparation_page()
    elif pathname == '/estadisticas':
        return get_estadistic_page()
    else:  # Incluyendo '/'
        return get_home_page()

In [357]:
@app.callback(
    Output('map-container', 'children'),
    [Input('lat-long-radio-items', 'value'),  # Escucha los cambios de latitud y longitud
    Input('zoom-position-radio-items', 'value')]  # Escucha los cambios de zoom y posición
)

def update_map(lat_long_value, zoom_position_value):
    # Aquí deberías decidir qué mapa mostrar basado en los valores seleccionados
    # Este es un pseudocódigo, necesitarás ajustar la lógica según tus datos y necesidades específicas
    if lat_long_value == 3:
        minx, miny, maxx, maxy = solicitar_coordenadas(gdf)
        # Define aquí cómo generas el mapa para la Opción 3
        map_image = get_map_image_as_html(map_ilustration(gdf, minx, miny, maxx, maxy), map_heat(gdf, minx, miny, maxx, maxy, bin_count, posicion_x, posicion_y, zoom))
    elif lat_long_value == 2:
        minx=-8.689
        miny=41.107
        maxx=-8.560
        maxy=41.185
        # Define aquí cómo generas el mapa para la Opción 2
        map_image = get_map_image_as_html(map_ilustration(gdf, minx, miny, maxx, maxy), map_heat(gdf, minx, miny, maxx, maxy, bin_count, posicion_x, posicion_y, zoom))
    else:
        # Define aquí cómo generas el mapa para la Opción 1
        map_image = get_map_image_as_html(html_map, html_heatmap)

    return [map_image]

In [358]:
@app.callback(
    Output('map-clusters-1', 'children'),
    [Input('item-1-1', 'n_clicks'),
    Input('item-1-2', 'n_clicks'),
    Input('item-1-3', 'n_clicks')]
)

def display_clusters_1(n_clicks):
    ctx = dash.callback_context

    if not ctx.triggered:
        return "Seleccione un elemento para el mapa 1."
    else:
        button_id = ctx.triggered[0]['prop_id'].split('.')[0]
        if button_id == 'item-1-1':
            return get_clusters_map()
        elif button_id == 'item-1-2':
            return get_clusters_map()
        elif button_id == 'item-1-3':
            return get_clusters_map()


In [359]:
@app.callback(
    Output('map-clusters-2', 'children'),
    [Input('item-2-1', 'n_clicks'),
    Input('item-2-2', 'n_clicks'),
    Input('item-2-3', 'n_clicks')]
)

def display_clusters_2(n_clicks):
    ctx = dash.callback_context

    if not ctx.triggered:
        return "Seleccione un elemento para el mapa 2."
    else:
        button_id = ctx.triggered[0]['prop_id'].split('.')[0]
        if button_id == 'item-2-1':
            return get_clusters_map()
        elif button_id == 'item-2-2':
            return get_clusters_map()
        elif button_id == 'item-2-3':
            return get_clusters_map()

In [360]:
# Ejecutar la aplicación
if __name__ == '__main__':
    app.run_server(debug=True, host='127.0.0.1', port=8050)
    # http://127.0.0.1:8050/